# Python Data Analysis for Radiotherapy Applications

## Contents:
* Dose map with gammas and protons in external RT
* Dose map analysis for internal RT
* PhSp files: reading IEAE and ROOT formats
* GateTools

# Dose map analysis with the example Ex_dose3D

#### We will first have a look at a GATE example to produce a 3D dose map:
* open a terminal and go to the folder gate_radiotherapy/Ex_dose3D
* look at the different files and open mac/main.mac
* look at all the sections and in particular to the geometry and ouputs section
* run the example with  
`Gate --qt mac/main.mac`
* check the output folder and the different files produced
* analyse the output 3D dose map with the following cells  
(some high statistic results with 1e6 particles are available in results.1e6/)

In [ ]:
cd Ex_dose3D/

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cat main.mac

In [ ]:
!Gate main.mac

In [ ]:
import numpy as np
import SimpleITK as sitk
import ipywidgets as ipw
# to enable interactive widgets in jupyter notebook run this command in a terminal:
#jupyter nbextension enable --py widgetsnbextension
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import uproot


In [ ]:
# Read the patient CT with sitk
img_ct = sitk.ReadImage('data/patient-2mm.mhd')

print('Image size: ', img_ct.GetSize())
print('Image spacing: ', img_ct.GetSpacing())
print('Image origin: ', img_ct.GetOrigin())

In [ ]:
# Convert sitk image to a numpy array
arr_ct = sitk.GetArrayFromImage(img_ct)
print('Array size: ', arr_ct.shape, ' <--- be careful to the dimension order!)')

In [ ]:


# function to display 3D image slices
def show_ct(sx,sy,sz):
    fig, ax = plt.subplots(1, 3, figsize=(16, 8)) # Create an array of axes : 1 row, 3 columns
    ax[0].imshow(arr_ct[sx, :, :], origin='lower', cmap='bone', vmin=arr_ct.min(), vmax=arr_ct.max()*0.7)
    ax[1].imshow(arr_ct[:, sy, :], origin='lower', cmap='bone', vmin=arr_ct.min(), vmax=arr_ct.max()*0.7)
    ax[2].imshow(arr_ct[:, :, sz], origin='lower', cmap='bone', vmin=arr_ct.min(), vmax=arr_ct.max()*0.7)
    plt.show()
    
ipw.interact(show_ct, sx=(0,arr_ct.shape[0]-1), sy=(0,arr_ct.shape[1]-1), sz=(0,arr_ct.shape[2]-1));

In [ ]:
# read the GATE output result
img_dose = sitk.ReadImage('output/3d-Dose.mhd')
# img_dose = sitk.ReadImage('Ex_dose3D/results.1e6/3d-gamma-Dose.mhd')
arr_dose = sitk.GetArrayFromImage(img_dose)
print('Image size = ', arr_dose.shape)
print('Image min and max: ', np.min(arr_dose), np.max(arr_dose))

In [ ]:
# display the output result
def show_dose(nslice):
    plt.figure()
    plt.imshow(arr_dose[:, :, nslice], cmap='hot', vmin=0, vmax=arr_dose.max())
    plt.colorbar()
    plt.show()
    
ipw.interact(show_dose, nslice=(0, arr_dose.shape[2]-1));

In [ ]:
# resample the dose map to match the CT resolution
img_resampled_dose = sitk.Resample(img_dose, img_ct, sitk.Transform(), sitk.sitkLinear, 0)
arr_resampled_dose = sitk.GetArrayFromImage(img_resampled_dose)
print('Image size = ', arr_resampled_dose.shape)
print('Image min and max: ',  np.min(arr_resampled_dose), np.max(arr_resampled_dose))

In [ ]:
# function to overlay the CT and the dose map
def show_fusion(nslice=64, opacity=0.7):
    min_dose_disp = 1e-9
    plt.figure(figsize=(10, 7))
    plt.imshow(arr_ct[:, :, nslice], vmin=arr_ct.min(), vmax=arr_ct.max(), cmap='bone')
    a = arr_resampled_dose[:, :, nslice]
    b = np.ma.masked_where(a <= min_dose_disp, a)
    plt.imshow(b, alpha=opacity, cmap='hot', vmin=min_dose_disp, vmax=arr_resampled_dose.max())
    plt.colorbar()
    plt.show()
    
ipw.interact(show_fusion, nslice=(0, arr_ct.shape[2]-1), opacity=(0, 1, 0.1));

## Exercise
* make a depth dose plot
* change the particle type to 100 MeV protons in the main.mac
* run the exemple again and analyse the results
* explain the difference with gammas

In [ ]:
p_edep = arr_resampled_dose.sum(axis=(2, 1))
print(p_edep.shape)

plt.figure()
plt.plot(p_edep)
plt.show()

# Dose map analysis with the example Ex_internal-RT

## Exercise
* open a terminal and go to the folder gate_radiotherapy/Ex_Internal-RT
* look at the different files and open mac/main.mac
* run the example with  
`Gate --qt mac/main.mac`
* use this notebook to analyse the 3D dose map result (from the "dose map analysis" section)  
(some high statistic results with 1e6 particles are available in results.1e6/)
* display a slice of the 3D data with plt.imshow
* display a slice of the SPECT image with plt.imshow
* use the SPECT activity matrix to select a region in the dose map region and get the total aborbed dose in that region

In [ ]:
cd ../Ex_InternalRT/

In [ ]:
pwd

In [ ]:
!Gate main.mac

In [ ]:
# read the GATE output result


In [ ]:
# function to display 3D image slices


In [ ]:
# read the GATE output result


In [ ]:
# display the output result


In [ ]:
# function to overlay the CT and the dose map


In [ ]:
# read the SPECT image


In [ ]:
# apply a selection on the dose map


# Phase-space (PhSp) analysis

# Reading IAEA phase-space files
You can get IAEA phase-space files from the IAEA database: https://www-nds.iaea.org/phsp/phsp.htmlx
* CyberKnife_IRIS
* ELEKTA_Precise
* SIEMENS_Primus
* Varian_Clinac
* Varian_TrueBeam

# Example with Varian phase-space file

In [ ]:
cd ../

In [ ]:
pwd

In [ ]:

# For PHOTON and ELECTRON Varian Native File
dt = np.dtype([('p', np.int8), ('e', np.float32), ('x', np.float32), ('y', np.float32), ('z', np.float32), ('dx', np.float32), ('dy', np.float32)])

# For PHOTON an ELECTRON phase space actor output file (GATE) 
#dt=np.dtype([('p', np.int8), ('e', np.float32), ('x', np.float32), ('y', np.float32), ('z', np.float32), ('dx', np.float32), ('dy', np.float32), ('we', np.float32), ('inc', np.float32) ])

# Test on Reduced PhSp from the IAEA online database
data = np.fromfile('data/Varian_TrueBeam6MV_sample.IAEAphsp', dtype=dt)
print('Number of particles:', len(data))
# data

In [ ]:
# Test on Reduced PhSp from the IAEA online database (first particles)
data=np.fromfile('data/Varian_TrueBeam6MV_sample.IAEAphsp', dtype=dt, count=800)
print('Number of particles:', len(data))

In [ ]:
#to save the data to a file
new_data = bytearray(data)
with open("output/Varian_TrueBeam6MV_test.IAEAphsp", "wb") as file:
    file.write(new_data)
    
# Or you can just
data.tofile("output/Varian_TrueBeam6MV_test.IAEAphsp")    


In [ ]:


# Plot the energy histogram
x = data['e']

plt.figure()
n, bins, patches = plt.hist(x, bins=200, alpha=0.75)
plt.show()

In [ ]:
# Plot the particle positions
x = data['x']
y = data['y']

plt.figure()
# fig = plt.scatter(x, y, alpha=0.75)
# fig = plt.plot(x, y, '.', alpha=0.75)
fig = plt.hist2d(x, y, bins=50, cmap='hot')
plt.colorbar()
plt.show()

# print(fig)

## Exercise
* save the 1000 particles splitted in 10 different files
* find a method to save the numpy array directly to a file

# PhSp analysis with the example Ex_PhSp
Example based on the exercise **linac** from the GATE exercices:
* Documentation https://davidsarrut.pages.in2p3.fr/gate-exercices-site/docs/exercice4/
* Source https://gitlab.in2p3.fr/davidsarrut/gate-exercices/-/tree/master/linac

## Exercise
* open a terminal and go to the folder Ex_PhSp/
* look at the different files and open the different main*.mac
* run the example with the different main macros  
`Gate mac/main-write-PhS.mac`
* check the output folder and the different files produced  
(some high statistic results are available in results.MD6k/)


* open the ROOT output output-PhS-g.root
* read the tree 'PhaseSpace' and get the data
(see notebook gate_outputs.ipynb for help)
* select and plot the histogram of the particle energies
* select and plot the particle positions
* select and plot the histogram of the particle theta angle

In [ ]:
cd Ex_PhSp/

In [ ]:
pwd

In [ ]:
!Gate main-write-PhS.mac

In [ ]:
# complete here

In [ ]:

f = uproot.open('output/output-PhS-g.root')
# f = uproot.open('results.MD6k/output-PhS-g.root')

# all trees, branches and leaves names are accessible through the method keys()
print('Trees in the file:')
print(f.keys())
print()
print('Leaves (variables) in the Tree:')
print(f['PhaseSpace'].keys())

data = f[f.keys()[0]].arrays(library="pd").to_records(index=False)
print(data)
print('Number of particles:', len(data))
print(data.dtype)

In [ ]:
# Plot the energy histogram


In [ ]:
# Plot the x,y positions of the particles


In [ ]:
# Plot the theta angle (main direction dZ)


# GATE tools
https://github.com/OpenGATE/GateTools

**Tools for [GATE](https://github.com/OpenGATE/Gate/) simulations.**

Install with : `pip install gatetools`

Example of usage:
```
gt_gate_info
gt_image_convert -i input.dcm -o output.mhd
gt_image_convert -i input.mhd -o output_float.mhd -p float
gt_image_arithm -i *.mhd -o output.mhd -O sum
gt_gamma_index dose.mhd gate-DoseToWater.mhd -o gamma.mhd --dd 2 --dta 2.5 -u "%" -T 0.2
```

Use the flag `-h` to get print the help of each tool. Here is the current list of command line tools.

| Tool                          | Comment                                                   |
| -------------                 | -------------                                             |
| `gt_dicom_rt_pbs2gate`        | Convert Dicom RT proton plan for Gate                     |
| `gt_dicom_rt_struct_to_image` | Turn Dicom RT Struct contours into mask image             |
| `gt_gamma_index`              | Compute gamma index between images                        |
| `gt_gate_info`                | Display info about current Gate/G4 version                |
| `gt_image_arithm`             | Pixel- or voxel-wise arithmetic operations                |
| `gt_image_convert`            | Convert image file format (**dicom**, mhd, hdr, nii ... ) |
| `gt_image_crop`               | Crop an image                                             |
| `gt_image_uncertainty`        | Compute statistical uncertainty                           |
| `gt_phsp_convert`             | Convert a phase space file from root to npy               |
| `gt_phsp_info`                | Display information about a phase space file              |
| `gt_phsp_merge`               | Merge two phase space files (output in npy only)          |
| `gt_phps_peaks`               | Try to detect photopeaks (experimental)                   |
| `gt_phsp_plot`                | Plot marginal distributions form a phase space file       |

All tools are also available to be use within your own Python script with, for example:
```
import gatetools as gt
gt.image_convert(inputImage, pixeltype)
```

Tests: run
```
python -m unittest gatetools -v
python -m unittest gatetools.phsp -v
```

Classes and function documentation. Use the following command to open a browser for documentation (it is still not very convenient ; will be improved later).
```
pydoc -b
```

For developers, please have a look at the [readme_dev.md](readme_dev.md) file.